In [1]:
from fastai.vision import *
from sklearn.model_selection import StratifiedKFold
from fastai.callbacks import *
from fastai.tabular import *

In [2]:
np.random.seed(4)






In [3]:
all_images_YPR_dlib_csv_file_base_path = \
        '/disks/data/paper_projects/selfie_detection/data/DS_224_all_features_no_duplicates/CSVs/im_skel_im-skel_mask_dlib-images_ypr_dlib_pts/'

testset_csv_save_path = '/disks/data/paper_projects/selfie_detection/data/DS_224_all_features_no_duplicates/CSVs/with_test_set/'

In [4]:
class_names = ['selfie', 'non_selfie', 'mirror_selfie', 'non_mirror_selfie']

#class_names = ['selfie', 'non_selfie']

In [5]:
class_dataframes = {}

for class_name in class_names:
    csv = pd.read_csv(all_images_YPR_dlib_csv_file_base_path+class_name+'.csv')
    csv.insert(0, column = 'class_label',value = len(csv.index)*[f"{class_name}"])
    
    class_dataframes[class_name] = csv
    
csv.head()

,class_label,im_name,x_min,y_min,x_max,y_max,confidence_score,yaw,pitch,roll,...,y_67,x_68,y_68,skeleton,im_skeleton,im_masked,dlib,im_dlib,skel_dlib,im_skel_dlib
0,non_mirror_selfie,34037497165_753a8f22e9_c.jpg,133.279892,29.287485,179.421066,84.647400,NaN,-16.768814,-20.375259,-2.652145,...,57,122,57,34037497165_753a8f22e9_c.jpg_skeleton.jpg,34037497165_753a8f22e9_c.jpg_im_skeleton.jpg,34037497165_753a8f22e9_c.jpg_masked.jpg,34037497165_753a8f22e9_c_dlib_only.jpg,34037497165_753a8f22e9_c_dlib.jpg,34037497165_753a8f22e9_c.jpg_skeleton_dlib.jpg,34037497165_753a8f22e9_c.jpg_im_skeleton_dlib.jpg
1,non_mirror_selfie,22934864204_b9337938d8_c.jpg,108.219429,49.803162,164.869629,118.680145,NaN,-10.305779,-25.460968,2.286850,...,80,100,80,22934864204_b9337938d8_c.jpg_skeleton.jpg,22934864204_b9337938d8_c.jpg_im_skeleton.jpg,22934864204_b9337938d8_c.jpg_masked.jpg,22934864204_b9337938d8_c_dlib_only.jpg,22934864204_b9337938d8_c_dlib.jpg,22934864204_b9337938d8_c.jpg_skeleton_dlib.jpg,22934864204_b9337938d8_c.jpg_im_skeleton_dlib.jpg
2,non_mirror_selfie,27540988553_ff93b68c1a_c.jpg,98.837723,34.895699,191.731400,139.520050,NaN,-24.857277,-6.494133,0.433289,...,84,121,84,27540988553_ff93b68c1a_c.jpg_skeleton.jpg,27540988553_ff93b68c1a_c.jpg_im_skeleton.jpg,27540988553_ff93b68c1a_c.jpg_masked.jpg,27540988553_ff93b68c1a_c_dlib_only.jpg,27540988553_ff93b68c1a_c_dlib.jpg,27540988553_ff93b68c1a_c.jpg_skeleton_dlib.jpg,27540988553_ff93b68c1a_c.jpg_im_skeleton_dlib.jpg
3,non_mirror_selfie,33880204342_a2ec399070_c.jpg,138.098938,59.103058,179.743134,106.467484,NaN,14.927979,-14.303543,26.582260,...,73,111,72,33880204342_a2ec399070_c.jpg_skeleton.jpg,33880204342_a2ec399070_c.jpg_im_skeleton.jpg,33880204342_a2ec399070_c.jpg_masked.jpg,33880204342_a2ec399070_c_dlib_only.jpg,33880204342_a2ec399070_c_dlib.jpg,33880204342_a2ec399070_c.jpg_skeleton_dlib.jpg,33880204342_a2ec399070_c.jpg_im_skeleton_dlib.jpg
4,non_mirror_selfie,26330164925_407599f41c_c.jpg,130.079346,47.824070,212.411896,126.606598,NaN,3.935005,-19.922783,13.808563,...,79,115,78,26330164925_407599f41c_c.jpg_skeleton.jpg,26330164925_407599f41c_c.jpg_im_skeleton.jpg,26330164925_407599f41c_c.jpg_masked.jpg,26330164925_407599f41c_c_dlib_only.jpg,26330164925_407599f41c_c_dlib.jpg,26330164925_407599f41c_c.jpg_skeleton_dlib.jpg,26330164925_407599f41c_c.jpg_im_skeleton_dlib.jpg


# Balancing dataset

In [6]:
available_nums = [len(class_dataframes[class_name].index) for class_name in class_names]
available_nums, min(available_nums) 

([1788, 1249, 1171, 1335], 1171)

In [7]:
sample_sz_min = min(available_nums)
min_i = np.argmin(available_nums)
sample_sz = [ np.random.randint(sample_sz_min, sample_sz_min + 50) for _ in class_names]
sample_sz[min_i] = sample_sz_min
sample_sz

[1217, 1176, 1171, 1211]

In [8]:
balanced_class_dfs = {class_name:class_dataframes[class_names[class_idx]].sample(n = sample_sz[class_idx], replace=False, random_state=4, axis=0) for class_idx, class_name in enumerate(class_names)}

In [9]:
balanced_class_df_sizes = [class_df.shape[0] for _,class_df in balanced_class_dfs.items()]

[class_df.shape for _,class_df in balanced_class_dfs.items()]

[(1217, 157), (1176, 157), (1171, 157), (1211, 157)]

In [10]:
test_set_pct = 0.2
test_set_sz = int(sample_sz_min*test_set_pct)
test_set_sz

234

In [11]:
test_set_idxs = [ np.array(sorted(( np.random.choice(np.arange(balanced_class_df_sizes[i]), size = test_set_sz, replace = False) ))) for i,class_name in enumerate(balanced_class_dfs.keys())]
[len(test_set_idx) for test_set_idx in test_set_idxs]

[234, 234, 234, 234]

In [12]:
# add an is_test column to the class' data frames
for i, class_name in enumerate(balanced_class_dfs.keys()):
    df = balanced_class_dfs[class_name]
    df = df.reset_index(drop = True)
    is_test_vals = np.array([0]*len(df.index))
    is_test_vals[test_set_idxs[i]] = 1
    #print(len(is_test_vals))
    #print(max(test_set_idxs[i]))
    #print(len(set(test_set_idxs[i])))
    #print(len(df.index))
    #print(np.sum(is_test_vals==1))
    df['is_test']= pd.Series(is_test_vals)
    balanced_class_dfs[class_name] = df
    #df.is_test.iloc[test_set_idxs[i]] = 1
    #print(sum(is_test_vals))
    #break

In [13]:
for i, class_name in enumerate(balanced_class_dfs.keys()):
    df = balanced_class_dfs[class_name]

    df_without_testset = df[df.is_test==0].copy()
    df_without_testset = df_without_testset.reset_index(drop = True)

    df_only_testset = df[df.is_test==1].copy()
    df_only_testset = df_only_testset.reset_index(drop = True)
    
    
    df.to_csv(testset_csv_save_path+f'{class_name}_with_testet.csv',
            columns = df.columns, 
            header = df.columns,
            index = False)
    
    df_without_testset.to_csv(testset_csv_save_path+f'{class_name}_without_testet.csv',
        columns = df_without_testset.columns, 
        header = df_without_testset.columns,
        index = False)

    df_only_testset.to_csv(testset_csv_save_path+f'{class_name}_only_testet.csv',
        columns = df_only_testset.columns, 
        header = df_only_testset.columns,
        index = False)    
    